# Question Answering System Using NLP


# Sample demo

#Question — “To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?”


#Context — “Architecturally, the school has a Catholic character. Atop the Main Building\’s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend “Venite Ad Me Omnes”. Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.”


#Text — “Saint Bernadette Soubirous”

Question answering system invovles two types


# knowledge-based and information retrieval

Knowledge based is fetching from the given text data
Information retrieval fetches through various documents though wikipedia by using an API developed by wikipedia


we will build an IR-based question answering system. First, we transform the question into a search query. Next, we use the MediaWiki API to get the documents. Finally, we extract the answers from the documents using a pre-trained BERT. 

# Steps Of action

The entire system can be divided into four parts: 






Query Processing

Document retrieval

Passage retrieval

Answer extraction

Accuracy

Web Interface implementation Using Flask

# Query Processing

Stop words are removed and words with specific part-of-speech are removed.and a method of converting a question into a vector that can express the meaning of a sentence well

for example

# the question “Who is the founder of Amazon?” generates the query “founder Amazon”

In [1]:
import spacy
import nltk
import string
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sra1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
text="Who is the founder of Amazon?"
#text= str(input())
text

'Who is the founder of Amazon?'

In [3]:
words=word_tokenize(text)
words

['Who', 'is', 'the', 'founder', 'of', 'Amazon', '?']

In [4]:
punctuations = list(string.punctuation)

words = [word for word in words if word not in punctuations]
print( "words without stopwords and punctuations:", words)


words without stopwords and punctuations: ['Who', 'is', 'the', 'founder', 'of', 'Amazon']


In [5]:
from nltk.corpus import stopwords
print(stopwords.words('english'))
stop_words = stopwords.words('english')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
words = [word for word in words if word not in stop_words]
print(len(words), "without stopwords:", words)

3 without stopwords: ['Who', 'founder', 'Amazon']


In [7]:
pos= nltk.pos_tag(words)
pos

[('Who', 'WP'), ('founder', 'NN'), ('Amazon', 'NNP')]

In [8]:
#proper nouns (PROPN),numbers (NUM), verbs (VERB), nouns (NOUN), and adjectives (ADJ) 

In [9]:
words=[s for s in pos if s[1] == 'NNP' or s[1] == 'NUM' or s[1] == 'NN' or s[1] == 'VERB' or s[1] == 'ADJ' ]

In [10]:
final=[]

In [11]:
for i in range( len(words)):
    final.append(words[i][0])

In [12]:
final

['founder', 'Amazon']

In [13]:
final_query=' '.join(final)
final_query



'founder Amazon'

# Document Retrieval


In the document retrieval, we will retrieve relevant documents by using the generated query. In the subsequent processing, since the answer will be extracted from these documents, it is required to search the documents that may contain the answer as much as possible. Similar to the question processing, deep learning technology allows us to convert documents into vectors

In [14]:
import wikipedia


In [15]:
page_list=wikipedia.search(final_query)

In [16]:
for i in page_list:
    print(i)

Amazon (company)
The Founder
Amazons
History of Amazon
Amazon Go
Amazon Web Services
Amazon Robotics
Jamie Siminoff
Amazon Game Studios
Jeff Bezos


In [17]:
docs=[]
for i in page_list:
    docs.append(wikipedia.page(i).content)
    

In [18]:
docs[0]

'Amazon.com, Inc. ( AM-ə-zon) is an American multinational technology company based in Seattle, Washington, which focuses on e-commerce, cloud computing, digital streaming, and artificial intelligence. It is considered one of the Big Four companies in the U.S. information technology industry, along with Google, Apple, and Facebook. The company has been referred to as "one of the most influential economic and cultural forces in the world", as well as the world\'s most valuable brand.Amazon was founded by Jeff Bezos in Bellevue, Washington, on July 5, 1994. The company started as an online marketplace for books but expanded to sell electronics, software, video games, apparel, furniture, food, toys, and jewelry. In 2015, Amazon surpassed Walmart as the most valuable retailer in the United States by market capitalization. In 2017, Amazon acquired Whole Foods Market for US$13.4 billion, which substantially increased its footprint as a physical retailer. In 2018, Bezos announced that its two

In [19]:
import torch

In [20]:
import concurrent.futures
import itertools
import operator
import re
import torch
import requests
import wikipedia
from gensim.summarization.bm25 import BM25
#from transformers import AutoTokenizer, AutoModel
#from transformers import AutoTokenizer, AutoModel, QuestionAnsweringPipeline
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering,QuestionAnsweringPipeline
#AutoModel

In [21]:
url='https://en.wikipedia.org/w/api.php'
params = {
            'action': 'query',
            'list': 'search',
            'srsearch': final_query,
            'format': 'json'
        }
res = requests.get(url, params=params)
print(res)

<Response [200]>


In [22]:
class QueryProcessor:

    def __init__(self, nlp, keep=None):
        self.nlp = nlp
        self.keep = keep or {'PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'}

    def generate_query(self, text):
        doc = self.nlp(text)
        query = ' '.join(token.text for token in doc if token.pos_ in self.keep)
        print(query)
        print('________________')
        return query


In [23]:
import en_core_web_sm
nlp = en_core_web_sm.load()

In [24]:
import os
SPACY_MODEL = os.environ.get('SPACY_MODEL', 'en_core_web_sm')
QA_MODEL = os.environ.get('QA_MODEL', 'distilbert-base-cased-distilled-squad')

In [25]:
nlp = spacy.load(SPACY_MODEL, disable=['ner', 'parser', 'textcat'])
query_processor = QueryProcessor(nlp)

In [26]:
question = str(input())

who is the father of India


In [27]:
query = query_processor.generate_query(question)

father India
________________


In [28]:
class DocumentRetrieval:

    def __init__(self, url='https://en.wikipedia.org/w/api.php'):
        self.url = url

    def search_pages(self, query):
        params = {
            'action': 'query',
            'list': 'search',
            'srsearch': query,
            'format': 'json'
        }
        res = requests.get(self.url, params=params)
        return res.json()

    def search_page(self, page_id):
        res = wikipedia.page(pageid=page_id)
        print(res.content)
        print('_____________')
        return res.content

    def search(self, query):
        pages = self.search_pages(query)
        with concurrent.futures.ThreadPoolExecutor() as executor:
            process_list = [executor.submit(self.search_page, page['pageid']) for page in pages['query']['search']]
            docs = [self.post_process(p.result()) for p in process_list]
        print(docs)
        return docs

    def post_process(self, doc):
        pattern = '|'.join([
            '== References ==',
            '== Further reading ==',
            '== External links',
            '== See also ==',
            '== Sources ==',
            '== Notes ==',
            '== Further references ==',
            '== Footnotes ==',
            '=== Notes ===',
            '=== Sources ===',
            '=== Citations ===',
        ])
        p = re.compile(pattern)
        indices = [m.start() for m in p.finditer(doc)]
        min_idx = min(*indices, len(doc))
        print(doc[:min_idx])
        return doc[:min_idx]


In [29]:
document_retriever = DocumentRetrieval()

In [30]:
docs = document_retriever.search(query)

The Father of the Nation is an honorific title given to a person considered the driving force behind the establishment of his/her country, state, or nation. Pater Patriae (plural Patres Patriae), also seen as Parens Patriae, was a Roman honorific meaning the "Father of the Fatherland", bestowed by the Senate on heroes, and later on emperors. In monarchies, the monarch was often considered the "father/mother of the nation" or as a patriarch to guide his family. This concept is expressed in the Divine Right espoused in some monarchies, while in others it is codified into constitutional law as in Spain, where the monarch is considered the personification and embodiment, the symbol of the unity and permanence of the nation. In Thailand, the monarch is given the same recognition, and demonstrated loyalty is enforced with severe criminal statutes.
Many dictators bestow titles upon themselves, which rarely survive the end of their regime. Gnassingbé Eyadéma of Togo's titles included "father o

The Indian cricket team are two times World Champions. In addition to winning the 1983 Cricket World Cup, they triumphed over Sri Lanka in the 2011 Cricket World Cup on home soil. They were also runners-up at the 2003 Cricket World Cup, and semifinalists four times(1987, 1996, 2015, 2019). India's historical win-loss record at the cricket world cup is 53-29, with 1 match being tied and another one being abandoned due to rain.


== Overall record ==
White: Group/round-robin stage
Green: Quarter-finals/Super Six
Light Blue: Semi-finals
Silver: Runners-up
Gold: Champions


=== By opponent ===


== India at the 1975 World Cup ==

The 1975 Cricket World Cup was the first Cricket World Cup. It was held in England in June 1975 and consisted of two weeks of one-day matches played 60-overs-a-side. The format consisted of a group stage, in which each team played the other three teams in its group of four. The top two teams from both groups would progress to the semifinals. India competed in Grou

Original text of the preamble
_____________
The following is a list of significant men and women known for being the father, mother, or founder (or similar) of a field in India. In most non-science fields, the title of being the "father" is debatable.


== List ==


== References ==
_____________
The following is a list of significant men and women known for being the father, mother, or founder (or similar) of a field in India. In most non-science fields, the title of being the "father" is debatable.


== List ==



Mohandas Karamchand Gandhi (; 2 October 1869 – 30 January 1948) was an Indian lawyer, anti-colonial nationalist, and political ethicist, who employed nonviolent resistance to lead the successful campaign for India's independence from British rule, and in turn inspired movements for civil rights and freedom across the world. The honorific Mahātmā (Sanskrit: "great-souled", "venerable"), first applied to him in 1914 in South Africa, is now used throughout the world. His contr

India is the second most populated country in the world with nearly a fifth of the world's population. According to the 2019 revision of the World Population Prospects population stood at 1,352,642,280.
During 1975–2010, the population doubled to 1.2 billion. The Indian population reached the billion mark in 1998. India is projected to be the world's most populous country by 2024, surpassing the population of China. It is expected to become the first country to be home to more than 1.5 billion people by 2030, and its population is set to reach 1.7 billion by 2050. Its population growth rate is 1.13%, ranking 112th in the world in 2017.India has more than 50% of its population below the age of 25 and more than 65% below the age of 35. It is expected that, in 2020, the average age of an Indian will be 29 years, compared to 37 for China and 48 for Japan; and, by 2030, India's dependency ratio should be just over 0.4. However, the number of children in India peaked more than a decade ago a

The Indian Independence Movement was a series of historic events with the ultimate aim of ending the British rule in India. The movement spanned from 1857 to 1947.The first nationalistic revolutionary movement for Indian independence emerged from Bengal. It later took root in the newly formed Indian National Congress with prominent moderate leaders seeking only their fundamental right to appear for Indian Civil Service examinations in British India, as well as more rights (economical in nature), for the people of the soil. The early part of the 20th century saw a more radical approach towards political self-rule proposed by leaders such as the Lal, Bal, Pal triumvirate, and Aurobindo Ghosh, V. O. Chidambaram Pillai.The last stages of the self-rule struggle from the 1920s was characterized by Congress's adoption of Mohandas Karamchand Gandhi's policy of non-violence and civil disobedience, and several other campaigns. Nationalists like Subhash Chandra Bose, Bhagat Singh, Bagha Jatin, Su

Yashwant Bhimrao Ambedkar (12 December 1912 - 17 September 1977), also known as Bhaiyasaheb Ambedkar, was an Indian socio-religious activist, newspaper editor, politician, and activist of Ambedkarite Buddhist movement. He was the first and only surviving child of Ramabai Ambedkar and Babasaheb Ambedkar, Indian polymath, human rights activist, and the father of the Constitution of India. Yashwant devoted his life to Buddhism after the demise of his father and kept pace his father's struggle for social equality. He tried to keep the Ambedkarite community united and also took an active part in the Dalit Buddhist movement.After his father passed away in 1956, he become the second president of the Buddhist Society of India and continued his father's struggle. In 1968, he organized an All India Buddhist Conference. After his death, his wife Mira became the president of the Buddhist Society of India. He had four children, including Prakash Yashwant Ambedkar.
He had been the editor of the "Jan

# Passage Retrieval

In [31]:
class PassageRetrieval:

    def __init__(self, nlp):
        self.tokenize = lambda text: [token.lemma_ for token in nlp(text)]
        self.bm25 = None
        self.passages = None

    def preprocess(self, doc):
        passages = [p for p in doc.split('\n') if p and not p.startswith('=')]
        return passages

    def fit(self, docs):
        passages = list(itertools.chain(*map(self.preprocess, docs)))
        corpus = [self.tokenize(p) for p in passages]
        self.bm25 = BM25(corpus)
        self.passages = passages

    def most_similar(self, question, topn=10):
        tokens = self.tokenize(question)
        scores = self.bm25.get_scores(tokens)
        pairs = [(s, i) for i, s in enumerate(scores)]
        pairs.sort(reverse=True)
        passages = [self.passages[i] for _, i in pairs[:topn]]
        print(passages)
        return passages


In [32]:
passage_retriever = PassageRetrieval(nlp)
passage_retriever.fit(docs)
passages = passage_retriever.most_similar(question)

['The following is a list of significant men and women known for being the father, mother, or founder (or similar) of a field in India. In most non-science fields, the title of being the "father" is debatable.', "Yashwant Bhimrao Ambedkar (12 December 1912 - 17 September 1977), also known as Bhaiyasaheb Ambedkar, was an Indian socio-religious activist, newspaper editor, politician, and activist of Ambedkarite Buddhist movement. He was the first and only surviving child of Ramabai Ambedkar and Babasaheb Ambedkar, Indian polymath, human rights activist, and the father of the Constitution of India. Yashwant devoted his life to Buddhism after the demise of his father and kept pace his father's struggle for social equality. He tried to keep the Ambedkarite community united and also took an active part in the Dalit Buddhist movement.After his father passed away in 1956, he become the second president of the Buddhist Society of India and continued his father's struggle. In 1968, he organized 

# Answer Extraction

In [33]:
class AnswerExtractor:

    def __init__(self, tokenizer, model):
        tokenizer = AutoTokenizer.from_pretrained(tokenizer)
        model = TFAutoModelForQuestionAnswering.from_pretrained(model)
        self.nlp = QuestionAnsweringPipeline(model=model, tokenizer=tokenizer)

    def extract(self, question, passages):
        answers = []
        for passage in passages:
            try:
                answer = self.nlp(question=question, context=passage)
                answer['text'] = passage
                answers.append(answer)
            except KeyError:
                pass
        answers.sort(key=operator.itemgetter('score'), reverse=True)
        print(answers)
        return answers

In [34]:
answer_extractor = AnswerExtractor(QA_MODEL, QA_MODEL)
answers = answer_extractor.extract(question, passages)

All model checkpoint weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-cased-distilled-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


[{'score': 0.8738973736763, 'start': 548, 'end': 569, 'answer': 'Sheikh Mujibur Rahman', 'text': 'In postcolonial Africa, "father of the nation" was a title used by many leaders both to refer to their role in the independence movement as a source of legitimacy, and to use paternalist symbolism as a source of continued popularity. On Joseph Stalin\'s seventieth birthday in 1949, he was bestowed with the title "Father of Nations" for his establishment of "people\'s democracies" in countries occupied by the USSR after World War II.The title "Father of the Nation" is sometimes politically contested. The 1972 Constitution of Bangladesh declared Sheikh Mujibur Rahman to be "father of the nation". The BNP government removed this in 2004, to the protests of the opposition Awami League, led by Rahman\'s daughter Sheikh Hasina. A motion in the Parliament of Slovakia to proclaim controversial pre-war leader Andrej Hlinka "father of the nation" barely failed in September 2007.'}, {'score': 0.75762

# Web Interface implementation Using Flask

In [36]:
import os

import spacy
from flask import Flask, render_template, jsonify, request

app = Flask(__name__)
SPACY_MODEL = os.environ.get('SPACY_MODEL', 'en_core_web_sm')
QA_MODEL = os.environ.get('QA_MODEL', 'distilbert-base-cased-distilled-squad')
nlp = spacy.load(SPACY_MODEL, disable=['ner', 'parser', 'textcat'])
query_processor = QueryProcessor(nlp)
document_retriever = DocumentRetrieval()
passage_retriever = PassageRetrieval(nlp)
answer_extractor = AnswerExtractor(QA_MODEL, QA_MODEL)


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/answer-question', methods=['POST'])
def analyzer():
    data = request.get_json()
    question = data.get('question')

    query = query_processor.generate_query(question)
    docs = document_retriever.search(query)
    passage_retriever.fit(docs)
    passages = passage_retriever.most_similar(question)
    answers = answer_extractor.extract(question, passages)
    return jsonify(answers)

app.run()

C:\ProgramData\Anaconda3\lib\site-packages\spacy\util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.2). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
Some weights of the model checkpoint at distilbert-base-cased-distilled-squad were not used when initializing TFDistilBertForQuestionAnswering: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from the checkpoint of a model that you expect to be e

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
